# Image Classification Model - CIFAR-10 Dataset
## Minor Project: Model Improvement Exercise

This notebook contains a baseline image classification model using the CIFAR-10 dataset. Your goal is to improve the model's performance through various techniques.

## 1. Setup and Imports

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## 2. Load and Explore the Dataset

CIFAR-10 consists of 60,000 32x32 color images in 10 classes:
- Airplane, Automobile, Bird, Cat, Deer, Dog, Frog, Horse, Ship, Truck

In [ ]:
# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Class names
class_names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 
               'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

print(f"Training data shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Test labels shape: {y_test.shape}")
print(f"\nNumber of classes: {len(class_names)}")
print(f"Pixel value range: [{X_train.min()}, {X_train.max()}]")

In [ ]:
# Visualize sample images
plt.figure(figsize=(15, 8))
for i in range(20):
    plt.subplot(4, 5, i + 1)
    plt.imshow(X_train[i])
    plt.title(class_names[y_train[i][0]], fontsize=10)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Check class distribution
unique, counts = np.unique(y_train, return_counts=True)
plt.figure(figsize=(12, 5))
plt.bar([class_names[i] for i in unique], counts, color='steelblue')
plt.xlabel('Classes', fontsize=12)
plt.ylabel('Number of Images', fontsize=12)
plt.title('Class Distribution in Training Set', fontsize=14, fontweight='bold')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Data Preprocessing

In [ ]:
# Normalize pixel values to [0, 1]
X_train_normalized = X_train.astype('float32') / 255.0
X_test_normalized = X_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train_categorical = to_categorical(y_train, num_classes=10)
y_test_categorical = to_categorical(y_test, num_classes=10)

print(f"Normalized training data shape: {X_train_normalized.shape}")
print(f"One-hot encoded labels shape: {y_train_categorical.shape}")
print(f"Sample label (original): {y_train[0]}")
print(f"Sample label (one-hot): {y_train_categorical[0]}")

## 4. Baseline Model (Simple CNN)

**Current Architecture:**
- 2 Convolutional layers with max pooling
- Flatten layer
- 1 Dense layer
- Output layer

**Areas for Improvement:**
1. Add more convolutional layers
2. Add batch normalization
3. Add dropout for regularization
4. Experiment with different optimizers
5. Use data augmentation
6. Try transfer learning (VGG, ResNet, etc.)
7. Adjust learning rate and batch size

In [ ]:
def create_baseline_model():
    """
    Creates a simple baseline CNN model.
    This is intentionally basic to give room for improvement.
    """
    model = models.Sequential([
        # First convolutional block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'),
        layers.MaxPooling2D((2, 2)),
        
        # Second convolutional block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        
        # Dense layers
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    
    return model

# Create the baseline model
baseline_model = create_baseline_model()
baseline_model.summary()

## 5. Compile and Train the Baseline Model

In [ ]:
# Compile the model
baseline_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'best_baseline_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

In [ ]:
# Train the model
history = baseline_model.fit(
    X_train_normalized,
    y_train_categorical,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

## 6. Evaluate Baseline Model Performance

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].legend(loc='lower right')
axes[0].grid(alpha=0.3)

# Loss plot
axes[1].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].legend(loc='upper right')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = baseline_model.evaluate(X_test_normalized, y_test_categorical, verbose=0)

print(f"\n{'='*50}")
print(f"BASELINE MODEL PERFORMANCE")
print(f"{'='*50}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"{'='*50}\n")

In [ ]:
# Get predictions
y_pred = baseline_model.predict(X_test_normalized)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = y_test.flatten()

# Classification report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=class_names))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Baseline Model', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Visualize some predictions
def plot_predictions(images, true_labels, pred_labels, class_names, num_samples=12):
    """
    Plot sample images with their true and predicted labels.
    """
    plt.figure(figsize=(15, 10))
    for i in range(num_samples):
        plt.subplot(3, 4, i + 1)
        plt.imshow(images[i])
        
        true_label = class_names[true_labels[i]]
        pred_label = class_names[pred_labels[i]]
        
        color = 'green' if true_labels[i] == pred_labels[i] else 'red'
        plt.title(f"True: {true_label}\nPred: {pred_label}", 
                 fontsize=10, color=color, fontweight='bold')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Plot some predictions
plot_predictions(X_test[:12], y_true_classes[:12], y_pred_classes[:12], class_names)

## 7. Improved Model Section

**TODO: Your task is to improve upon the baseline model.**

### Suggested Improvements:

1. **Architecture Improvements:**
   - Add more convolutional layers (3-5 blocks)
   - Increase number of filters progressively (32 → 64 → 128 → 256)
   - Add batch normalization after each conv layer
   - Add dropout layers (0.2-0.5) to prevent overfitting
   - Try residual connections (ResNet-style)

2. **Data Augmentation:**
   ```python
   from tensorflow.keras.preprocessing.image import ImageDataGenerator
   
   datagen = ImageDataGenerator(
       rotation_range=15,
       width_shift_range=0.1,
       height_shift_range=0.1,
       horizontal_flip=True,
       zoom_range=0.1
   )
   ```

3. **Transfer Learning:**
   - Use pre-trained models: VGG16, ResNet50, EfficientNet
   - Fine-tune the last few layers

4. **Hyperparameter Tuning:**
   - Learning rate: Try 0.001, 0.0001
   - Batch size: Try 32, 128, 256
   - Optimizers: Adam, SGD with momentum, RMSprop

5. **Advanced Techniques:**
   - Learning rate scheduling (cosine annealing)
   - Mixed precision training
   - Ensemble methods

### Implementation Space:

In [ ]:
# TODO: Create your improved model here

def create_improved_model():
    """
    Create an improved version of the baseline model.
    
    Implement your improvements here!
    """
    
    # Example: A more sophisticated architecture
    model = models.Sequential([
        # First block
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.2),
        
        # Second block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        
        # Third block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.4),
        
        # Dense layers
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    
    return model

# Uncomment to create and view the improved model
# improved_model = create_improved_model()
# improved_model.summary()

In [ ]:
# TODO: Implement data augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    fill_mode='nearest'
)

# Visualize augmented images
# sample_image = X_train_normalized[0:1]
# plt.figure(figsize=(15, 3))
# plt.subplot(1, 6, 1)
# plt.imshow(sample_image[0])
# plt.title('Original')
# plt.axis('off')

# i = 2
# for batch in datagen.flow(sample_image, batch_size=1):
#     plt.subplot(1, 6, i)
#     plt.imshow(batch[0])
#     plt.title(f'Augmented {i-1}')
#     plt.axis('off')
#     i += 1
#     if i > 6:
#         break
# plt.tight_layout()
# plt.show()

In [ ]:
# TODO: Compile and train your improved model

# Example compilation
# improved_model.compile(
#     optimizer=keras.optimizers.Adam(learning_rate=0.001),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

# Example training with data augmentation
# history_improved = improved_model.fit(
#     datagen.flow(X_train_normalized, y_train_categorical, batch_size=64),
#     epochs=50,
#     validation_data=(X_test_normalized, y_test_categorical),
#     callbacks=[early_stopping, reduce_lr, checkpoint],
#     verbose=1
# )

## 8. Compare Models

In [ ]:
# TODO: Evaluate and compare your improved model with the baseline

# Example evaluation
# test_loss_improved, test_accuracy_improved = improved_model.evaluate(
#     X_test_normalized, y_test_categorical, verbose=0
# )

# print(f"\n{'='*60}")
# print(f"MODEL COMPARISON")
# print(f"{'='*60}")
# print(f"Baseline Model - Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
# print(f"Improved Model - Test Accuracy: {test_accuracy_improved:.4f} ({test_accuracy_improved*100:.2f}%)")
# print(f"Improvement: {(test_accuracy_improved - test_accuracy)*100:.2f}%")
# print(f"{'='*60}\n")

## 9. Transfer Learning (Advanced)

For even better performance, try using pre-trained models:

In [ ]:
# TODO: Implement transfer learning (Optional)

# Example using EfficientNetB0
# from tensorflow.keras.applications import EfficientNetB0
# from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

# def create_transfer_learning_model():
#     # Load pre-trained model
#     base_model = EfficientNetB0(
#         include_top=False,
#         weights='imagenet',
#         input_shape=(32, 32, 3)
#     )
#     
#     # Freeze base model layers
#     base_model.trainable = False
#     
#     # Add custom classification head
#     inputs = keras.Input(shape=(32, 32, 3))
#     x = base_model(inputs, training=False)
#     x = GlobalAveragePooling2D()(x)
#     x = Dense(256, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     outputs = Dense(10, activation='softmax')(x)
#     
#     model = keras.Model(inputs, outputs)
#     return model

# transfer_model = create_transfer_learning_model()
# transfer_model.summary()

## 10. Save Your Best Model

In [ ]:
# Save the final model
# baseline_model.save('final_baseline_model.h5')
# improved_model.save('final_improved_model.h5')

# Or save in TensorFlow SavedModel format
# baseline_model.save('saved_models/baseline_model')
# improved_model.save('saved_models/improved_model')

## 11. Conclusion and Next Steps

### Summary of Improvements:
- Document what improvements you tried
- Note which techniques worked best
- Record your final accuracy improvements

### Future Work:
1. Try ensemble methods (combine multiple models)
2. Experiment with different architectures (DenseNet, ResNet, Vision Transformers)
3. Use techniques like mixup or cutmix
4. Apply neural architecture search (NAS)
5. Test on other datasets (CIFAR-100, ImageNet subset)

### Resources:
- [TensorFlow Documentation](https://www.tensorflow.org/)
- [Keras Applications](https://keras.io/api/applications/)
- [Papers with Code](https://paperswithcode.com/) - Latest SOTA models
- [Fast.ai](https://www.fast.ai/) - Practical deep learning